## Imports

In [1]:
#!/usr/bin/env python

import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from modules.myfuncs import *


## Query SQLite3 db into Pandas


In [2]:
conn = create_connection('database/rlc.db')  # function from myfuncs file
c = conn.cursor()


sqlite3 version: 2.6.0
connected to database/rlc.db


In [3]:
sql_fetch_tables(c, conn)

[('cam_startend',), ('cam_locations',), ('intersection_locations',), ('daily_violations',), ('intersection_cams',), ('signal_crashes',)]


In [9]:
crash_df = pd.read_sql_query("SELECT * FROM signal_crashes;", conn)

print(len(all_df))
crash_df.head()

camloc_df = pd.read_sql_query('SELECT * FROM intersection_cams', conn)
camloc_df.head()

60328


,intersection,n_cams,latitude,longitude,camera_id1,camera_id2,camera_id3
0,31ST ST AND MARTIN LUTHER KING DRIVE,2,41.780231,-87.615640,2121,2123,None
1,ASHLAND AND DIVERSEY,2,41.932484,-87.668562,1623,1621,None
2,IRVING PARK AND LARAMIE,2,41.953866,-87.716609,1533,1534,None
3,ELSTON AND IRVING PARK,2,41.953838,-87.720427,1503,1502,None
4,IRVING PARK AND KILPATRICK,2,41.953866,-87.716609,2764,2763,None


In [10]:
# What's in this data?
for col in ['traffic_control_device', 'intersection_related_i']:
    print(col, crash_df[col].unique())

traffic_control_device ['TRAFFIC SIGNAL']
intersection_related_i ['Y']


intersection_related_i: A field observation by the police officer whether an intersection played a role in the crash. Does not represent whether or not the crash occurred within the intersection.

In [11]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="https://github.com/sciencelee/chicago_rlc")


location = geolocator.geocode("175 5th Avenue NYC")
print(location.address)
# out: Flatiron Building, 175, 5th Avenue, Flatiron, New York, NYC, New York, ...

print((location.latitude, location.longitude))
# out: (40.7410861, -73.9896297241625)

#print(location.raw)
# out: {'place_id': '9167009604', 'type': 'attraction', ...}


# CAN USE THIS TO FIGURE OUT MY LAT LONG FROM RLC ADDRESS (or crash)


Flatiron Building, 175, 5th Avenue, Flatiron District, Manhattan Community Board 5, Manhattan, New York County, New York, 10010, United States
(40.741059199999995, -73.98964162240998)


In [12]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="https://github.com/sciencelee/chicago_rlc")


location = geolocator.geocode("4700 W IRVING PARK ROAD, CHICAGO")  # NS then EW
print(location.address)
# out: Flatiron Building, 175, 5th Avenue, Flatiron, New York, NYC, New York, ...

print((location.latitude, location.longitude))
# out: (40.7410861, -73.9896297241625)

#print(location.raw)
# out: {'place_id': '9167009604', 'type': 'attraction', ...}


# CAN USE THIS TO FIGURE OUT MY LAT LONG FROM RLC ADDRESS (or crash)

West Irving Park Road, Irving Park, Chicago, Jefferson Township, Cook County, Illinois, 60618, United States
(41.9538663, -87.7166086)


In [62]:
# Example from geopy docs
from geopy import distance
# newport_ri = (41.49008, -71.312796)
# cleveland_oh = (41.499498, -81.695391)
# print(distance.distance(newport_ri, cleveland_oh).miles)

# my example
rlc1 = (camloc_df.iloc[0]['latitude'], camloc_df.iloc[0]['longitude'])
print(camloc_df.iloc[0]['intersection'])
rlc2 = (camloc_df.iloc[11]['latitude'], camloc_df.iloc[11]['longitude'])
print(camloc_df.iloc[11]['intersection'])

print(rlc1, rlc2)
print(distance.distance(rlc1, rlc2).meters, 'meters')
print(distance.distance(rlc1, rlc2).feet, 'feet')


31ST ST AND MARTIN LUTHER KING DRIVE
ASHLAND AND IRVING PARK
(41.7802309, -87.6156396) (41.954491, -87.6694766843563)
19864.581642566893 meters
65172.51195067878 feet


In [65]:
# what is the closest rlc intersection to each accident.  Is it under 50m

def closest_rlc(camloc_df, lat, long, m):
    # m is meters distance threshold for determining at the correct intersection
    latlong = (lat, long)
    for i in range(len(camloc_df)):
        loc = ((camloc_df.iloc[i]['latitude'], camloc_df.iloc[i]['longitude']))
        if distance.distance(loc, latlong).meters < m:
            return camloc_df.iloc[i]['intersection']
            #print(loc, camloc_df.iloc[i]['intersection'], distance.distance(loc, latlong).meters)

    else:
        return None

#print(crash_df.iloc[1, :])
#closest_rlc(camloc_df, crash_df.iloc[1, :], 50)
crash_df['intersection'] = crash_df.apply(lambda x: closest_rlc(camloc_df, x.latitude, x.longitude, 30), axis=1)



KeyboardInterrupt: 

In [61]:
print(crash_df.intersection[:50])

AttributeError: 'DataFrame' object has no attribute 'intersection'